<div style="background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%); padding: 25px; border-radius: 10px; box-shadow: 0 4px 15px rgba(0,0,0,0.3); color: white; text-align: center; font-family: 'Segoe UI', sans-serif; margin-bottom: 20px;">
<h1 style="margin: 0; font-size: 2.2em; font-weight: 600;">Commodity Risk Management</h1>
<h2 style="margin: 10px 0 0 0; font-size: 1.4em; font-weight: 300; opacity: 0.9;">Asymmetric Volatility Forecasting & Dynamic VaR</h2>
<p style="margin-top: 5px; font-size: 1em; font-style: italic; opacity: 0.7;">GJR-GARCH Approach</p>
</div>

<div style="background-color: rgba(255, 255, 255, 0.05); border-left: 5px solid #4ec9b0; padding: 15px 20px; border-radius: 4px; font-family: sans-serif; line-height: 1.6; color: inherit;">
<p style="margin-bottom: 10px;">
<strong>📊 The Core Observation:</strong> ARCH/GARCH models are built on the idea that volatility spikes suddenly and persists before reverting to its long-term mean. This creates a phenomenon known as <em>volatility clustering</em>.
</p>
<p style="margin-bottom: 10px;">
<strong>⚙️ Mechanics:</strong> In the GARCH framework:
<br>• <i>u</i><sub>t-1</sub><sup>2</sup> represents the <strong>shock</strong> generating volatility.
<br>• The &beta; (Beta) coefficient captures the <strong>persistence</strong> of volatility after a shock.
</p>
<p style="margin-bottom: 0; font-size: 0.9em; opacity: 0.8;">
<em>Extensions included: Exponential GARCH, asymmetric models, regime-switching, etc.</em>
</p>
</div>

In [ ]:
%matplotlib inline

<h2 style="
    border-bottom: 2px solid #4ec9b0; 
    padding-bottom: 10px; 
    margin-top: 30px; 
    margin-bottom: 20px; 
    font-family: 'Segoe UI', sans-serif; 
    color: white;">
    <span style="color: #4ec9b0; font-weight: bold; margin-right: 10px;">01.</span>
    Data Extraction & Preparation
</h2>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [ ]:
data = yf.download('BNO', start = "2015-01-01", end = '2025-11-27')

data.columns = data.columns.droplevel(1)

data

In [ ]:
prices = data.Close

returns = np.log(prices/prices.shift(1))

data['Returns'] = returns

data

In [ ]:
data = data.dropna()

data

<h2 style="
    border-bottom: 2px solid #4ec9b0; 
    padding-bottom: 10px; 
    margin-top: 30px; 
    margin-bottom: 20px; 
    font-family: 'Segoe UI', sans-serif; 
    color: white;">
    <span style="color: #4ec9b0; font-weight: bold; margin-right: 10px;">02.</span>
    Descriptive Analysis & Visualization
</h2>

In [ ]:
returns.plot(figsize=(16,9), color = 'red')
plt.xlabel('Date')
plt.ylabel('Returns')
plt.grid(True)
plt.show()

<div style="background-color: rgba(255, 255, 255, 0.05); border-left: 5px solid #4ec9b0; padding: 20px; border-radius: 4px; font-family: 'Segoe UI', sans-serif; line-height: 1.6; color: inherit;">
<p style="margin-bottom: 15px; font-style: italic; opacity: 0.9;">Visual inspection of the Brent log-return series reveals <strong>four key stylized facts</strong>, justifying the choice of econometric methods employed hereafter:</p>
<div style="margin-bottom: 15px;">
<strong style="color: #4ec9b0;">1. Stationarity & Mean Reversion 🔄</strong><br>
Unlike the raw price series which exhibits a stochastic trend, the return series oscillates around a constant mean close to zero. This stationarity is crucial: it indicates that statistical properties (mean, variance) do not linearly depend on time, validating the potential use of classical linear models (ARMA) for the conditional mean.
</div>
<div style="margin-bottom: 15px;">
<strong style="color: #4ec9b0;">2. Volatility Clustering (Heteroscedasticity) 〰️</strong><br>
The plot perfectly illustrates the phenomenon theorized by Mandelbrot: <em>"Large changes tend to be followed by large changes, and small by small."</em> We observe distinct alternating regimes:
<ul style="margin: 5px 0 5px 20px; list-style-type: disc;">
<li>Periods of relative calm (e.g., 2017) with low variance.</li>
<li>Periods of high turbulence (e.g., 2020, 2022) with high-amplitude shocks.</li>
</ul>
This conditional heteroscedasticity violates the assumption of constant variance (homoscedasticity), making <strong>ARCH/GARCH</strong> models indispensable.
</div>
<div style="margin-bottom: 15px;">
<strong style="color: #4ec9b0;">3. Impact of Exogenous Shocks 💥</strong><br>
The series bears the signature of major geopolitical and sanitary crises. The extreme negative spike in early 2020 corresponds to the <strong>COVID-19 oil crash</strong> (approx. -25% drop). Similarly, the volatility surge in 2022 coincides with the Ukraine conflict and global inflationary tensions. These events confirm the asset's sensitivity to macro shocks.
</div>
<div>
<strong style="color: #4ec9b0;">4. Leptokurtosis & "Fat Tails" 🔔</strong><br>
The distribution significantly deviates from a Normal (Gaussian) Law. The presence of numerous outliers beyond standard confidence intervals (±0.05 or ±0.10) evidences

In [ ]:
(returns ** 2).plot(figsize=(16,9), color = 'red')
plt.xlabel('Date')
plt.ylabel('Squared Returns')
plt.grid(True)
plt.show()

<div style="background-color: rgba(255, 255, 255, 0.05); border-left: 5px solid #4ec9b0; padding: 20px; border-radius: 4px; font-family: 'Segoe UI', sans-serif; line-height: 1.6; color: inherit;">
<p style="margin-bottom: 15px;">
<strong>📉 Squared Returns (<i>r<sub>t</sub></i><sup>2</sup>)</strong> are essential because they constitute the best observable proxy for the <em>unobservable</em> conditional variance (volatility) of returns at a given time.
</p>
<p style="margin-bottom: 20px; font-size: 0.95em; opacity: 0.8;">
Since daily returns usually have a mean very close to zero (&mu; &approx; 0), the squared return acts as a direct approximation of the variance.
</p>
<div style="margin-bottom: 10px;">
<strong style="color: #4ec9b0;">1. Visualizing Volatility Clustering 👁️</strong><br>
They allow for a clear visualization of volatility clustering (regrouping of high-variance periods) which might be less obvious in raw returns.
</div>
<div style="margin-bottom: 10px;">
<strong style="color: #4ec9b0;">2. Model Input (Variance Equation) 📐</strong><br>
They serve as the dependent variable (the input) for the variance equation within the GARCH model structure.
</div>
<div>
<strong style="color: #4ec9b0;">3. Statistical Testing (ARCH-LM / Ljung-Box) 🧪</strong><br>
They are the subject of the <strong>Ljung-Box</strong> or <strong>ARCH-LM test</strong>. If <i>r<sub>t</sub></i><sup>2</sup> are significantly autocorrelated, it proves that volatility has temporal dependence and that a GARCH model is statistically necessary.
</div>
</div>

In [ ]:
import statsmodels
from statsmodels.stats.diagnostic import acorr_ljungbox

In [ ]:
returns_clean = returns.dropna()

In [ ]:
ljung_box = acorr_ljungbox(returns_clean**2., lags = 10, return_df = True)

print(ljung_box.apply(lambda x: x.map('{:.4f}'.format)))

<h2 style="border-bottom: 2px solid #4ec9b0; padding-bottom: 10px; margin-top: 30px; margin-bottom: 20px; font-family: 'Segoe UI', sans-serif; color: white;">
<span style="color: #4ec9b0; font-weight: bold; margin-right: 10px;">03.</span>
Conditional Mean Modeling
</h2>

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
p = 1
q = 1
model = ARIMA(returns,order=(p,0,q))
arma = model.fit()

print(arma.summary())

<div style="background-color: rgba(255, 255, 255, 0.05); border-left: 5px solid #4ec9b0; padding: 20px; border-radius: 4px; font-family: 'Segoe UI', sans-serif; line-height: 1.6; color: inherit;">
<p style="margin-bottom: 15px;">
<strong>Hypothesis Testing: The Zero-Mean Assumption</strong>
</p>
<div style="margin-bottom: 15px;">
<strong style="color: #4ec9b0;">1. Statistical Evidence 📉</strong><br>
The sample mean is extremely low (approx. 0.000009). More importantly, the <strong>p-value > 0.05</strong> implies that we fail to reject the null hypothesis (<i>H</i><sub>0</sub>: &mu; = 0). The mean is statistically indistinguishable from zero.
</div>
<div>
<strong style="color: #4ec9b0;">2. Modeling Implication 🎯</strong><br>
Since &mu; &approx; 0, we can simplify the conditional mean equation. The return <i>r<sub>t</sub></i> is approximately equal to the residual (innovation) term:
<br><br>
<center>
<span style="background-color: rgba(78, 201, 176, 0.1); padding: 5px 15px; border-radius: 4px; border: 1px solid #4ec9b0;">
<i>r<sub>t</sub></i> &approx; <i>&epsilon;<sub>t</sub></i>
</span>
</center>
<br>
This allows us to focus directly on modeling the variance (GARCH) without fitting a complex ARIMA model for the mean first.
</div>
</div>

In [ ]:
residuals = arma.resid

residuals_clean = residuals.dropna()

residuals_clean

<h2 style="border-bottom: 2px solid #4ec9b0; padding-bottom: 10px; margin-top: 30px; margin-bottom: 20px; font-family: 'Segoe UI', sans-serif; color: white;">
<span style="color: #4ec9b0; font-weight: bold; margin-right: 10px;">04.</span>
GARCH vs. GJR-GARCH Modeling
</h2>

In [ ]:
from arch import arch_model

am = arch_model(residuals_clean*100, mean = 'Constant', vol = 'Garch', p = 1, q = 1, dist='Normal' )

res_garch = am.fit(disp='off')

print(res_garch.summary())

In [ ]:
gjr = arch_model(residuals_clean*100, mean = 'Constant', vol = 'GARCH', p = 1, o = 1, q = 1, dist = 't')
res_gjr = gjr.fit(disp='off')
print(res_gjr.summary())

<div style="background-color: rgba(255, 255, 255, 0.05); border-left: 5px solid #4ec9b0; padding: 20px; border-radius: 4px; font-family: 'Segoe UI', sans-serif; line-height: 1.6; color: inherit;">
<h3 style="margin-top: 0; color: white; border-bottom: 1px solid rgba(255,255,255,0.1); padding-bottom: 10px;">
🏆 Model Selection: Statistical Comparison
</h3>
<table style="width: 100%; border-collapse: collapse; margin-top: 15px; margin-bottom: 20px; font-size: 0.95em;">
<tr style="border-bottom: 1px solid rgba(255,255,255,0.2); color: #4ec9b0;">
<th style="text-align: left; padding: 8px;">Metric</th>
<th style="text-align: center; padding: 8px;">Target</th>
<th style="text-align: center; padding: 8px;">GARCH(1,1)</th>
<th style="text-align: center; padding: 8px; background-color: rgba(78, 201, 176, 0.1); border-radius: 4px;">GJR-GARCH(1,1)</th>
</tr>
<tr>
<td style="padding: 8px; font-weight: bold;">Log-Likelihood</td>
<td style="text-align: center; font-size: 0.8em; opacity: 0.7;">(Maximize &uarr;)</td>
<td style="text-align: center;">-5864.30</td>
<td style="text-align: center; color: #4ec9b0; font-weight: bold; background-color: rgba(78, 201, 176, 0.1);">-5785.12</td>
</tr>
<tr>
<td style="padding: 8px; font-weight: bold;">AIC</td>
<td style="text-align: center; font-size: 0.8em; opacity: 0.7;">(Minimize &darr;)</td>
<td style="text-align: center;">11736.60</td>
<td style="text-align: center; color: #4ec9b0; font-weight: bold; background-color: rgba(78, 201, 176, 0.1);">11582.25</td>
</tr>
<tr>
<td style="padding: 8px; font-weight: bold;">BIC</td>
<td style="text-align: center; font-size: 0.8em; opacity: 0.7;">(Minimize &darr;)</td>
<td style="text-align: center;">11760.26</td>
<td style="text-align: center; color: #4ec9b0; font-weight: bold; background-color: rgba(78, 201, 176, 0.1);">11617.75</td>
</tr>
</table>
<div style="background-color: rgba(78, 201, 176, 0.1); padding: 15px; border-radius: 4px; border: 1px solid rgba(78, 201, 176, 0.3);">
<strong style="color: #4ec9b0;">✅ Conclusion:</strong>
The <strong>GJR-GARCH</strong> model is superior. It exhibits a higher <em>Log-Likelihood</em> (better fit) and lower Information Criteria (AIC & BIC), indicating that the inclusion of the asymmetry parameter justifies the slight increase in model complexity.
</div>
</div>

<h2 style="border-bottom: 2px solid #4ec9b0; padding-bottom: 10px; margin-top: 30px; margin-bottom: 20px; font-family: 'Segoe UI', sans-serif; color: white;">
<span style="color: #4ec9b0; font-weight: bold; margin-right: 10px;">05.</span>
Statistical Validation & Model Diagnostics
</h2>

<div style="background-color: rgba(255, 255, 255, 0.05); border-left: 5px solid #4ec9b0; padding: 20px; border-radius: 4px; font-family: 'Segoe UI', sans-serif; line-height: 1.6; color: inherit;">
<p style="margin-bottom: 15px;">
<strong>🔍 Diagnostics on Standardized Residuals (<i>z<sub>t</sub></i>)</strong>
</p>
<p style="margin-bottom: 15px; font-size: 0.95em; opacity: 0.8;">
To validate the GJR-GARCH model, we must inspect the standardized residuals, defined as:
<br>
<center>
<i>z<sub>t</sub></i> = <i>&epsilon;<sub>t</sub></i> / <i>&sigma;<sub>t</sub></i>
</center>
</p>
<div style="margin-bottom: 10px;">
<strong style="color: #4ec9b0;">1. No Autocorrelation (White Noise) 📉</strong><br>
The standardized residuals should behave like white noise. We use the <strong>Ljung-Box Test</strong> to confirm that no linear dependence remains.
</div>
<div style="margin-bottom: 10px;">
<strong style="color: #4ec9b0;">2. No Remaining ARCH Effects 🧬</strong><br>
We perform the <strong>ARCH-LM Test</strong> on squared standardized residuals (<i>z<sub>t</sub></i><sup>2</sup>) to ensure the model has successfully captured all volatility clustering.
</div>
<div>
<strong style="color: #4ec9b0;">3. Distribution Fit 🔔</strong><br>
We compare the distribution of <i>z<sub>t</sub></i> against the assumed distribution (Student-t) using a Q-Q Plot.
</div>
</div>

In [ ]:
residuals_gjr = res_gjr.std_resid

residuals_gjr_squared = residuals_gjr ** 2

ljung_box_gjr = acorr_ljungbox(residuals_gjr_squared, lags = 10, return_df= True)
print(ljung_box_gjr.apply(lambda x: x.map('{:.4f}'.format)))

<div style="background-color: rgba(255, 255, 255, 0.05); border-left: 5px solid #4ec9b0; padding: 20px; border-radius: 4px; font-family: 'Segoe UI', sans-serif; line-height: 1.6; color: inherit;">
<p style="margin-bottom: 15px;">
<strong style="font-size: 1.1em;">✅ Validation Success: Absence of ARCH Effects</strong>
</p>
<div style="margin-bottom: 15px;">
<strong style="color: #4ec9b0;">1. Statistical Evidence (p-values > 0.05) 📊</strong><br>
All p-values from the ARCH-LM test exceed the 5% significance level. Therefore, we cannot reject the Null Hypothesis (<i>H</i><sub>0</sub>: no ARCH effects).
</div>
<div style="margin-bottom: 15px;">
<strong style="color: #4ec9b0;">2. Interpretation 🧹</strong><br>
There is no remaining autocorrelation in the squared standardized residuals. The GJR-GARCH model has successfully <strong>"cleaned"</strong> all conditional heteroscedasticity from the data.
</div>
<div>
<strong style="color: #4ec9b0;">3. Conclusion 🏁</strong><br>
The residuals are now <strong>homoscedastic</strong> (they behave like white noise regarding variance). The model is statistically valid and robust for forecasting.
</div>
</div>

<h2 style="border-bottom: 2px solid #4ec9b0; padding-bottom: 10px; margin-top: 30px; margin-bottom: 20px; font-family: 'Segoe UI', sans-serif; color: white;">
<span style="color: #4ec9b0; font-weight: bold; margin-right: 10px;">06.</span>
Volatility Forecasting & Value-at-Risk (VaR)
</h2>

<div style="background-color: rgba(255, 255, 255, 0.05); border-left: 5px solid #4ec9b0; padding: 20px; border-radius: 4px; font-family: 'Segoe UI', sans-serif; line-height: 1.6; color: inherit;">
<p style="margin-bottom: 15px;">
<strong>🔮 From Volatility to Risk Management: Dynamic VaR</strong>
</p>
<div style="margin-bottom: 15px;">
<strong style="color: #4ec9b0;">1. The Concept 🛡️</strong><br>
Value-at-Risk (VaR) estimates the maximum potential loss over a specific time horizon at a given confidence level (e.g., 95%). Unlike static methods, our <strong>Dynamic VaR</strong> adjusts daily based on the GJR-GARCH volatility forecast.
</div>
<div style="margin-bottom: 15px;">
<strong style="color: #4ec9b0;">2. The Formula 📐</strong><br>
Since the mean is negligible (&mu; &approx; 0), the One-Day VaR at significance level <i>&alpha;</i> (e.g., 5%) is calculated as:
<br><br>
<center>
<span style="background-color: rgba(78, 201, 176, 0.1); padding: 5px 15px; border-radius: 4px; border: 1px solid #4ec9b0;">
VaR<sub>t</sub> = &sigma;<sub>t</sub> &times; <i>Q</i><sub>&alpha;</sub>
</span>
</center>
<br>
Where:
<ul style="margin-top: 5px; list-style-type: circle; padding-left: 20px;">
<li><strong>&sigma;<sub>t</sub></strong> : Conditional volatility forecast by GJR-GARCH.</li>
<li><strong><i>Q</i><sub>&alpha;</sub></strong> : The quantile of the Student-t distribution (capturing fat tails).</li>
</ul>
</div>
<div>
<strong style="color: #4ec9b0;">3. Interpretation 💡</strong><br>
If the actual return falls below the VaR line, it is considered a <strong>"VaR Breach"</strong> (or exception). This signals an extreme market event that the model predicted with only <i>&alpha;</i> probability.
</div>
</div>

In [ ]:
data_test = yf.download('BNO', start = '2025-11-27', end = '2025-12-05')

prices_test = data_test.Close

returns_test = np.log(prices_test/prices_test.shift(1))

data_test['Returns Test'] = returns_test

data_test

In [ ]:
import scipy.stats as stats

horizon_days = 5
forecasts = res_gjr.forecast(horizon=horizon_days)

pred_vol_series = np.sqrt(forecasts.variance.iloc[-1].values)

real_returns_series = data_test['Returns Test'].dropna()

limit = min(len(real_returns_series), horizon_days)

comparison = pd.DataFrame({
    'Date': real_returns_series.index[:limit],
    'Real Return (%)': real_returns_series.values[:limit] * 100,
    'Forecasted Volatility (%)': pred_vol_series[:limit]
})

nu = res_gjr.params['nu']
t_quantile = stats.t.ppf(0.01, nu)
comparison['VaR 99% (%)'] = comparison['Forecasted Volatility (%)'] * t_quantile

<div style="background-color: rgba(255, 255, 255, 0.05); border-left: 5px solid #4ec9b0; padding: 20px; border-radius: 4px; font-family: 'Segoe UI', sans-serif; line-height: 1.6; color: inherit;">
<h3 style="margin-top: 0; color: white; border-bottom: 1px solid rgba(255,255,255,0.1); padding-bottom: 10px;">
🏁 Final Test: Forecast vs. Reality (Backtest)
</h3>
<table style="width: 100%; border-collapse: collapse; margin-top: 15px; margin-bottom: 10px; font-size: 0.95em;">
<tr style="border-bottom: 1px solid rgba(255,255,255,0.2); color: #4ec9b0;">
<th style="text-align: left; padding: 10px;">Date</th>
<th style="text-align: right; padding: 10px;">Real Return (%)</th>
<th style="text-align: right; padding: 10px;">Forecasted Vol (%)</th>
<th style="text-align: right; padding: 10px;">VaR 99% (%)</th>
<th style="text-align: center; padding: 10px;">Status</th>
</tr>
<tr style="border-bottom: 1px solid rgba(255,255,255,0.05);">
<td style="padding: 8px;">2025-12-01</td>
<td style="text-align: right; color: #2ecc71;">+0.068 %</td>
<td style="text-align: right;">1.751 %</td>
<td style="text-align: right; color: #e74c3c; font-weight: bold;">-5.351 %</td>
<td style="text-align: center;">✅ Safe</td>
</tr>
<tr style="border-bottom: 1px solid rgba(255,255,255,0.05);">
<td style="padding: 8px;">2025-12-02</td>
<td style="text-align: right; color: #e74c3c;">-1.241 %</td>
<td style="text-align: right;">1.764 %</td>
<td style="text-align: right; color: #e74c3c; font-weight: bold;">-5.391 %</td>
<td style="text-align: center;">✅ Safe</td>
</tr>
<tr style="border-bottom: 1px solid rgba(255,255,255,0.05);">
<td style="padding: 8px;">2025-12-03</td>
<td style="text-align: right; color: #2ecc71;">+0.508 %</td>
<td style="text-align: right;">1.777 %</td>
<td style="text-align: right; color: #e74c3c; font-weight: bold;">-5.421 %</td>
<td style="text-align: center;">✅ Safe</td>
</tr>
<tr>
<td style="padding: 8px;">2025-12-04</td>
<td style="text-align: right; color: #2ecc71;">+0.721 %</td>
<td style="text-align: right;">1.709 %</td>
<td style="text-align: right; color: #e74c3c; font-weight: bold;">-5.465 %</td>
<td style="text-align: center;">✅ Safe</td>
</tr>
</table>
<div style="margin-top: 15px; font-size: 0.9em; opacity: 0.8; font-style: italic;">
* <strong>Safe</strong> implies that the Real Return > VaR 99%. No breach occurred during this period.
</div>
</div>

In [ ]:
import matplotlib.pyplot as plt

conditional_vol = res_gjr.conditional_volatility 

returns_hist = residuals_clean * 100 

plt.figure(figsize=(14, 7))

plt.plot(returns_hist.index, returns_hist, color='blue', alpha=0.4, lw=1, label='Brent Daily Returns')

plt.plot(conditional_vol.index, conditional_vol, color='red', lw=1.5, label='Estimated Volatility (GJR-GARCH)')
plt.plot(conditional_vol.index, -conditional_vol, color='red', lw=1.5)

plt.title('Brent Volatility Modeling : GJR-GARCH(1,1) Student', fontsize=16)
plt.ylabel('Returns / Volatility (%)', fontsize=12)
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)

plt.show()

<h2 style="border-bottom: 2px solid #4ec9b0; padding-bottom: 10px; margin-top: 30px; margin-bottom: 20px; font-family: 'Segoe UI', sans-serif; color: white;">
<span style="color: #4ec9b0; font-weight: bold; margin-right: 10px;">07.</span>
Backtesting & Final Conclusion
</h2>

<div style="background-color: rgba(255, 255, 255, 0.05); border-left: 5px solid #4ec9b0; padding: 20px; border-radius: 4px; font-family: 'Segoe UI', sans-serif; line-height: 1.6; color: inherit;">
<p style="margin-bottom: 15px;">
<strong>🏁 Final Test: Forecast vs. Reality</strong>
</p>
<div style="margin-bottom: 15px;">
<strong style="color: #4ec9b0;">1. Out-of-Sample Backtesting 📅</strong><br>
We compare the model's predictions (<i>ex-ante</i>) against the actual market data (<i>ex-post</i>) for the most recent trading days. This is the ultimate test of the model's predictive power.
</div>
<div style="margin-bottom: 15px;">
<strong style="color: #4ec9b0;">2. VaR Breach Check 🚨</strong><br>
We monitor if the <strong>Real Return</strong> falls below the <strong>VaR 99%</strong> threshold.
<ul style="margin-top: 5px; list-style-type: circle; padding-left: 20px;">
<li>If <i>Return</i> > <i>VaR</i>: The risk was correctly covered. ✅</li>
<li>If <i>Return</i> < <i>VaR</i>: It is a <strong>Breach</strong> (Exception). Too many breaches would invalidate the model.</li>
</ul>
</div>
<div>
<strong style="color: #4ec9b0;">3. Visual Fit 📈</strong><br>
The final plot illustrates the <strong>Vol-Envelope</strong>. The red line (Estimated Volatility) should tightly hug the blue spikes (Returns), expanding during crises and contracting during calm periods.
</div>
</div>

In [ ]:
comparison['VaR Validation'] = np.where(
    comparison['Real Return (%)'] < comparison['VaR 99% (%)'], 
    '❌ EXCEPTION',  
    '✅ OK'       
)

comparison['Score Sigma'] = comparison['Real Return (%)'] / comparison['Forecasted Volatility (%)']

<div style="background-color: rgba(255, 255, 255, 0.05); border-left: 5px solid #4ec9b0; padding: 20px; border-radius: 4px; font-family: 'Segoe UI', sans-serif; line-height: 1.6; color: inherit;">
<h3 style="margin-top: 0; color: white; border-bottom: 1px solid rgba(255,255,255,0.1); padding-bottom: 10px;">
🏁 Backtesting Results: VaR Breach Analysis
</h3>
<table style="width: 100%; border-collapse: collapse; margin-top: 15px; margin-bottom: 10px; font-size: 0.9em;">
<tr style="border-bottom: 1px solid rgba(255,255,255,0.2); color: #4ec9b0;">
<th style="text-align: left; padding: 10px;">Date</th>
<th style="text-align: right; padding: 10px;">Real Return</th>
<th style="text-align: right; padding: 10px;">Forecast Vol</th>
<th style="text-align: right; padding: 10px;">VaR 99%</th>
<th style="text-align: center; padding: 10px;">Score &sigma;</th>
<th style="text-align: center; padding: 10px;">Validation</th>
</tr>
<tr style="border-bottom: 1px solid rgba(255,255,255,0.05);">
<td style="padding: 8px; opacity: 0.8;">2025-12-01</td>
<td style="text-align: right; color: #2ecc71;">+0.069 %</td>
<td style="text-align: right;">1.752 %</td>
<td style="text-align: right; color: #e74c3c;">-5.351 %</td>
<td style="text-align: center; opacity: 0.7;">0.039</td>
<td style="text-align: center;"><span style="background-color: rgba(46, 204, 113, 0.2); color: #2ecc71; padding: 2px 8px; border-radius: 10px; font-size: 0.85em; border: 1px solid #2ecc71;">✅ OK</span></td>
</tr>
<tr style="border-bottom: 1px solid rgba(255,255,255,0.05);">
<td style="padding: 8px; opacity: 0.8;">2025-12-02</td>
<td style="text-align: right; color: #e74c3c;">-1.241 %</td>
<td style="text-align: right;">1.765 %</td>
<td style="text-align: right; color: #e74c3c;">-5.391 %</td>
<td style="text-align: center; opacity: 0.7;">-0.703</td>
<td style="text-align: center;"><span style="background-color: rgba(46, 204, 113, 0.2); color: #2ecc71; padding: 2px 8px; border-radius: 10px; font-size: 0.85em; border: 1px solid #2ecc71;">✅ OK</span></td>
</tr>
<tr style="border-bottom: 1px solid rgba(255,255,255,0.05);">
<td style="padding: 8px; opacity: 0.8;">2025-12-03</td>
<td style="text-align: right; color: #2ecc71;">+0.508 %</td>
<td style="text-align: right;">1.777 %</td>
<td style="text-align: right; color: #e74c3c;">-5.421 %</td>
<td style="text-align: center; opacity: 0.7;">0.331</td>
<td style="text-align: center;"><span style="background-color: rgba(46, 204, 113, 0.2); color: #2ecc71; padding: 2px 8px; border-radius: 10px; font-size: 0.85em; border: 1px solid #2ecc71;">✅ OK</span></td>
</tr>
<tr>
<td style="padding: 8px; opacity: 0.8;">2025-12-04</td>
<td style="text-align: right; color: #2ecc71;">+0.722 %</td>
<td style="text-align: right;">1.789 %</td>
<td style="text-align: right; color: #e74c3c;">-5.465 %</td>
<td style="text-align: center; opacity: 0.7;">0.403</td>
<td style="text-align: center;"><span style="background-color: rgba(46, 204, 113, 0.2); color: #2ecc71; padding: 2px 8px; border-radius: 10px; font-size: 0.85em; border: 1px solid #2ecc71;">✅ OK</span></td>
</tr>
</table>
<div style="margin-top: 10px; font-size: 0.85em; opacity: 0.8; font-style: italic;">
<strong>Definition:</strong> <em>Score &sigma;</em> measures the return standardized by the forecasted volatility. A score outside [-3, +3] would indicate an extreme event.
</div>
</div>

In [ ]:
import scipy.stats as stats
import numpy as np

volatility = res_gjr.conditional_volatility
nu = res_gjr.params['nu']
t_quantile = stats.t.ppf(0.01, nu)
VaR_series = volatility * t_quantile

def kupiec_pof_test(returns, var_series, confidence_level=0.99):
    alpha = 1 - confidence_level
    N = len(returns)
    breaches = returns < var_series
    x = np.sum(breaches)
    p = x / N

    if x == 0:
        print("Aucune brèche observée ! Le modèle est (trop) prudent.")
        return 0, 1.0, x, N
    
    numerator = ((1 - alpha)**(N - x)) * (alpha**x)
    denominator = ((1 - p)**(N - x)) * (p**x)
    
    lr_stat = -2 * np.log(numerator / denominator)
    p_value = 1 - stats.chi2.cdf(lr_stat, df=1)
    
    return lr_stat, p_value, x, N

lr, p_val, failures, total = kupiec_pof_test(residuals_clean * 100, VaR_series, confidence_level=0.99)

<div style="background-color: rgba(255, 255, 255, 0.05); border-left: 5px solid #e74c3c; padding: 20px; border-radius: 4px; font-family: 'Segoe UI', sans-serif; line-height: 1.6; color: inherit;">
<h3 style="margin-top: 0; color: #e74c3c; border-bottom: 1px solid rgba(231, 76, 60, 0.3); padding-bottom: 10px;">
❌ Kupiec Test Diagnosis: Model Validation Failed
</h3>
<p style="margin-bottom: 15px; font-style: italic; opacity: 0.9;">
The Kupiec test indicates that the model is <strong>too conservative</strong>. The observed number of exceptions (breaches) is significantly lower than the theoretical expectation for a 99% VaR.
</p>

<div style="margin-bottom: 20px; background-color: rgba(231, 76, 60, 0.1); padding: 15px; border-radius: 4px; border: 1px solid #e74c3c;">
<strong style="color: #e74c3c;">📉 The Diagnosis:</strong><br>
<ul style="margin: 5px 0 0 20px;">
<li><strong>Observed Failures:</strong> 0.62% (vs 1.00% expected).</li>
<li><strong>Implication:</strong> The calculated VaR overestimates the risk. It is "too high" and rarely breached.</li>
<li><strong>Result:</strong> Reject <i>H</i><sub>0</sub> (p-value 0.0315 < 0.05).</li>
</ul>
</div>

<div style="margin-bottom: 0;">
<strong style="color: white;">🛠️ Strategic Fixes (Recalibration Roadmap):</strong>
<ol style="margin-top: 10px; padding-left: 20px; color: opacity: 0.9;">
<li style="margin-bottom: 8px;"><strong>Parameter Tuning:</strong> Adjust GJR-GARCH coefficients or the Student-t degrees of freedom (DoF) to better reflect observed losses.</li>
<li style="margin-bottom: 8px;"><strong>VaR Scaling:</strong> Apply a slight multiplicative factor (< 1) to lower the VaR without altering confidence levels, aligning exceptions with theory.</li>
<li style="margin-bottom: 8px;"><strong>Distribution Selection:</strong> Adopt asymmetric distributions (e.g., Skewed Student-t) or semi-parametric methods (Cornish-Fisher) to better capture skewness and fat tails.</li>
<li><strong>Iterative Backtesting:</strong> Re-calculate and test iteratively until the model satisfies the Kupiec condition (p-value > 0.05).</li>
</ol>
</div>
</div>